In [ ]:
# Imports
import json
import os
import time

import akkio

import utils

In [ ]:
# Setup
API_KEY = "6bb5ca2b-ad1d-4eb4-b3a6-24cf1878dc63"
akkio.api_key = API_KEY
utils.API_KEY = API_KEY

In [ ]:
# Partition Data (OPTIONAL)
# filename = "ACI_Data_Small.csv"

# utils.partition_data(filename, out_location=os.getcwd(), test_partition=0.1, shuffle=True)

In [ ]:
# Import Data
train_filepath = os.path.join("data", "All Clients Model Train.csv")

train_data = utils.import_data(train_filepath)

train_data_dict = utils.df_to_dict(train_data)

In [ ]:
# Create dataset obj
dataset_name = "All_Client_Data_API_Test"

new_dataset = akkio.create_dataset(dataset_name)
print(new_dataset)

# get dataset_id
dataset_id = new_dataset["dataset_id"]

# Add rows to new dataset
response = utils.add_rows_to_dataset(dataset_id, train_data_dict)

time.sleep(5) # Needed currently to ensure dataset parsing completes on the backend

In [ ]:
# Train ML Model 
# (Train a dummy model to force project creation in the UI (I know this is suboptimal, but we will optimize this out in our v2 API))

# UI Mapping:
#     1    = Super Fast (Do not recommend)
#     10   = Fastest
#     60   = High Quality
#     300  = Higher Quality
#     1800 = Production

training_mode = 300
predict_field = ["Actual Impressions"]
ignore_fields = ["Actual Spend (USD)"]

dataset_id = "bzBgUgBcYzuD7R9sMh1F"

new_model = akkio.create_model(
    dataset_id,
    predict_field,
    ignore_fields,
    {"duration": training_mode, "extra_attention": True},
)

for key,val in new_model.items():
    print(key)
    print(val)
    print("\n")

In [ ]:
# Predictions
show_driving_factors = True
save_preds = True

resp_directory = "prediction_results"
os.makedirs(resp_directory, exist_ok=True)  # Ensure directory exists

test_filename = os.path.join("data","Test Dataset with Renormalisation_Test_Partition.csv") # path to CSV

test_data = utils.import_data(test_filename)
test_data_dict = utils.df_to_dict(test_data)

# get model_id
model_id = new_model["model_id"]

test_filepath = os.path.join(resp_directory, "client_impressions_predictions_PQ.csv")

# Need to overwrite make_prediction to pass in show_factors:true in json body of POST call -- add to utils and rename functions
prediction = utils.make_prediction(model_id, test_data_dict, show_factors=show_driving_factors, save=save_preds, save_file_path=test_filepath)
for pred in prediction['predictions']:
    print(pred)